In [1]:
##########################################
# This notebook is used for intial data exploration for the capstone project
##########################################


First, we import the data

In [102]:
import pandas as pd
import numpy as np
import glob
from IPython.display import display # Allows the use of display() for DataFrames


data = pd.read_csv('data/transactions_200607.csv')  #import one file

#import all csvs (courtesy of http://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe)

path =r'data' # use your path
allFiles = glob.glob(path + "/tr*.csv")

#print allFiles
#data = pd.DataFrame()
#list_ = []
#for file_ in allFiles:
#    df = pd.read_csv(file_,index_col=None, header=0)
#    list_.append(df)
#data = pd.concat(list_)

#print(data.head(20))


print('Before DropNA',len(data))
data=data.dropna()
print('After DropNA',len(data))

data = data[data.CUST_PRICE_SENSITIVITY != 'XX']
data = data[data.CUST_LIFESTAGE != 'OT']
print('After Uninteresting categories',len(data))
#list(data.columns.values)

('Before DropNA', 250546)
('After DropNA', 175047)
('After Uninteresting categories', 122605)


Reformat the data into a wide dataset using Prod_code_20 as the lowest level of aggregation.  

In [126]:
#create basketprofiles.  Simplest version will be to sum spend in each category in PROD_CODE_20, keeping the target variable as well
#pivot code from http://stackoverflow.com/questions/41046766/using-and-graphing-the-results-of-a-crosstab-dataframe-in-python

data_cross=data.pivot_table(index='BASKET_ID', columns='PROD_CODE_20', values='SPEND', aggfunc=np.sum, fill_value=0)
data_cross.reset_index(level=['BASKET_ID'], inplace=True)

#group the variables that are unique to each basket
bybasket=data.groupby(['BASKET_ID'])
targetsByBasket=pd.DataFrame(bybasket['CUST_LIFESTAGE', 'CUST_PRICE_SENSITIVITY'].first())
targetsByBasket.reset_index(level=['BASKET_ID'], inplace=True)

colsByBasketInt=pd.DataFrame(bybasket['SHOP_WEEKDAY', 'SHOP_HOUR'].first())
colsByBasketInt.reset_index(level=['BASKET_ID'], inplace=True)

colsByBasketStr=pd.DataFrame(bybasket['BASKET_SIZE', 'BASKET_TYPE','BASKET_DOMINANT_MISSION','STORE_FORMAT','STORE_REGION'].first())
colsByBasketStr.reset_index(level=['BASKET_ID'], inplace=True)
colsByBasketStr_dummied = pd.get_dummies(colsByBasketStr, prefix=['BASKET_SIZE', 'BASKET_TYPE','BASKET_DOMINANT_MISSION','STORE_FORMAT','STORE_REGION'])

sumsByBasket=pd.DataFrame(bybasket['SPEND'].sum())
sumsByBasket.reset_index(level=['BASKET_ID'], inplace=True)

data_cross_dummied = pd.merge(data_cross, sumsByBasket, how='inner', on = 'BASKET_ID')
data_cross_dummied = pd.merge(data_cross_dummied, colsByBasketInt, how='inner', on = 'BASKET_ID')
data_cross_dummied = pd.merge(data_cross_dummied, colsByBasketStr_dummied, how='inner', on = 'BASKET_ID')
data_cross_dummied = pd.merge(data_cross_dummied, targetsByBasket, how='inner', on = 'BASKET_ID')


data_cross = pd.merge(data_cross, sumsByBasket, how='inner', on = 'BASKET_ID')
data_cross = pd.merge(data_cross, colsByBasketInt, how='inner', on = 'BASKET_ID')
data_cross = pd.merge(data_cross, colsByBasketStr, how='inner', on = 'BASKET_ID')
data_cross = pd.merge(data_cross, targetsByBasket, how='inner', on = 'BASKET_ID')


#reset the index to what it should be for the rest
data_cross.set_index(['BASKET_ID'], inplace=True)
data_cross_dummied.set_index(['BASKET_ID'], inplace=True)

print(data_cross.head(5))
print(data_cross_dummied.head(5))
print(len(data_cross_dummied))
#list(data_cross.columns.values)

PROD_CODE_20     DEP00001  DEP00002  DEP00003  DEP00004  DEP00005  DEP00006  \
BASKET_ID                                                                     
994100100152667      0.00       0.0      0.00      0.00       0.0       0.0   
994100100152709      0.00       0.0      0.98      0.00       0.0       0.0   
994100100152710      1.03       0.0      0.00      0.00       0.0       0.0   
994100100152771      0.00       0.0      1.01      0.00       0.0       0.0   
994100100152772      2.25       0.0      0.00      0.56       0.0       0.0   

PROD_CODE_20     DEP00007  DEP00008  DEP00009  DEP00010  \
BASKET_ID                                                 
994100100152667       0.0      0.00       0.0       0.0   
994100100152709       0.0      1.17       0.0       0.0   
994100100152710       0.0      0.00       0.0       0.0   
994100100152771       0.0      0.00       0.0       0.0   
994100100152772       0.0      1.46       0.0       0.8   

PROD_CODE_20              ...   

# Naive Classifier Work

In [ ]:
#do all the work for the naive classifier

#create variables for the naive classifier
spendByBasket=pd.DataFrame(bybasket['SPEND'].sum())
spendByBasket.reset_index(level=['BASKET_ID'], inplace=True)

basketTypeByBasket=pd.DataFrame(bybasket['BASKET_TYPE'].first())
basketTypeByBasket.reset_index(level=['BASKET_ID'], inplace=True)

basketSizeByBasket=pd.DataFrame(bybasket['BASKET_SIZE'].first())
basketSizeByBasket.reset_index(level=['BASKET_ID'], inplace=True)

weekdayByBasket=pd.DataFrame(bybasket['SHOP_WEEKDAY'].first())
weekdayByBasket.reset_index(level=['BASKET_ID'], inplace=True)

#merge them back in
data_cross_naive = pd.merge(data_cross, spendByBasket, how='inner', on = 'BASKET_ID')
data_cross_naive = pd.merge(data_cross_naive, basketTypeByBasket, how='inner', on = 'BASKET_ID')
data_cross_naive = pd.merge(data_cross_naive, basketSizeByBasket, how='inner', on = 'BASKET_ID')
data_cross_naive = pd.merge(data_cross_naive, weekdayByBasket, how='inner', on = 'BASKET_ID')

In [15]:
#create the naive predictor

avgTrainSpend=X_train['SPEND'].mean()
stDevTrainSpend=X_train['SPEND'].std()


def NaiveClassifier(test, trainMean, trainStd):

    if(test['SPEND']<trainMean-trainStd):
            if(test['BASKET_TYPE']='Full Shop')
                   naivePriceSens = 'LA'
            else:
                    naivePriceSens = 'MM'
    elif(test['SPEND']>rainMean-trainStd):
        if(test['BASKET_TYPE']='Full Shop')
                   naivePriceSens = 'MM
            else:
                    naivePriceSens = 'UM'
    else:
        naivePriceSens = 'MM'
    
    if (test["SHOP_WEEKDAY"] = 7) or (test["SHOP_WEEKDAY"] = 1):
        if(test['BASKET_SIZE'] = 'Small'):
            if(test['BASKET_TYPE']='Small Shop')
                   naivePriceSens = 'YA'
            else:
                    naivePriceSens = 'OA'
        elif(test['BASKET_SIZE'] = 'Medium'):
            naiveLifeStage = 'YF'
        else:
            naiveLifeStage = 'OF'            
    else:
        naiveLifeStage = 'PE'
    
    
    
    return naivePriceSens + "-" naiveLifeStage

#apply to every row
#X_test.apply(NaiveClassifier, args=(avgTrainSpend,stDevTrainSpend)) # returns DataFrame


In [ ]:
#pop the two classifiers used for the naive predictor before doing any classification
X_train.pop("SPEND")
X_train.pop("SHOP_WEEKDAY")
X_test.pop("SPEND")
X_test.pop("SHOP_WEEKDAY")

# Split the data #should probably consider stratified sampling here.

In [114]:
#create the training, testing split.  can't use sklearn.cross_validation.train_test_split since I have two targets

#first do a 70-30 split.  
train_X=data_cross.sample(frac=0.7,random_state=42)
test_X=data_cross.drop(train_X.index)

#pop off the classifiers
train_y = train_X[["CUST_LIFESTAGE","CUST_PRICE_SENSITIVITY"]] #potentially for use in multi-output decision trees
train_y_LS = train_X.pop("CUST_LIFESTAGE")
train_y_PS = train_X.pop("CUST_PRICE_SENSITIVITY")

test_y = test_X[["CUST_LIFESTAGE","CUST_PRICE_SENSITIVITY"]] #potentially for use in multi-output decision trees
test_y_LS = test_X.pop("CUST_LIFESTAGE")
test_y_PS = test_X.pop("CUST_PRICE_SENSITIVITY")

#repeate for dummied data
#first do a 70-30 split.  
train_X_dummied=data_cross_dummied.sample(frac=0.7,random_state=42)
test_X_dummied=data_cross_dummied.drop(train_X_dummied.index)

#pop off the classifiers
train_y_dummied = train_X_dummied[["CUST_LIFESTAGE","CUST_PRICE_SENSITIVITY"]] #potentially for use in multi-output decision trees
train_y_LS_dummied = train_X_dummied.pop("CUST_LIFESTAGE")
train_y_PS_dummied = train_X_dummied.pop("CUST_PRICE_SENSITIVITY")

test_y_dummied = test_X_dummied[["CUST_LIFESTAGE","CUST_PRICE_SENSITIVITY"]] #potentially for use in multi-output decision trees
test_y_LS_dummied = test_X_dummied.pop("CUST_LIFESTAGE")
test_y_PS_dummied = test_X_dummied.pop("CUST_PRICE_SENSITIVITY")

print(len(train_X_dummied.columns))
print(len(test_X_dummied.columns))

print(len(train_y_dummied.columns))
print(len(test_y_dummied.columns))
#print(test_X.columns)

120
120
2
2


# Dimensionality Reduction TODO add indexes, column names and other columns to pca'd output

In [42]:
from sklearn.decomposition import PCA

#first, peel off the non-numerical variables

train_X_cat = train_X[['BASKET_SIZE', 'BASKET_TYPE','BASKET_DOMINANT_MISSION','STORE_FORMAT','STORE_REGION']]
test_X_cat = test_X[['BASKET_SIZE', 'BASKET_TYPE','BASKET_DOMINANT_MISSION','STORE_FORMAT','STORE_REGION']]

train_X_num = train_X.drop(['BASKET_SIZE', 'BASKET_TYPE','BASKET_DOMINANT_MISSION','STORE_FORMAT','STORE_REGION'], axis=1)
test_X_num = test_X.drop(['BASKET_SIZE', 'BASKET_TYPE','BASKET_DOMINANT_MISSION','STORE_FORMAT','STORE_REGION'], axis=1)


pca = PCA(n_components = 6)
pca.fit(train_X_num)

# TODO: Transform the sample log-data using the PCA fit above
train_X_pca= pca.transform(train_X_num)
test_X_pca= pca.transform(test_X_num)

#check cumulative sums
display(np.cumsum(pca.explained_variance_ratio_))



array([ 0.98441663,  0.99331894,  0.99428344,  0.99516127,  0.99563725,
        0.99602425])

# Downsample the training so all strata are equal

In [ ]:
probably need to code this myself, based on the particular target i want

# try the classifiers

In [48]:
rng = np.random.RandomState(1)
X = np.sort(200 * rng.rand(100, 1) - 100, axis=0)
y = np.array([np.pi * np.sin(X).ravel(), np.pi * np.cos(X).ravel()]).T
y[::5, :] += (0.5 - rng.rand(20, 2))



[[ 1.82569761  2.64490159]
 [ 2.80694332  1.41091233]
 [-2.72043225 -1.57125834]
 [-2.99372396 -0.95248164]
 [-0.85218806  3.02380223]
 [ 2.38902829 -1.33007108]
 [-2.79534903 -1.43374621]
 [-3.07394347  0.64844117]
 [-3.03515591  0.81082243]
 [ 0.46676469  3.10672418]
 [ 2.65732301  0.56229152]
 [ 2.63212356 -1.71508891]
 [ 2.28617935 -2.15475947]
 [-3.13109169 -0.25665006]
 [ 3.09796044  0.5217715 ]
 [ 1.25663563 -3.47458334]
 [-0.35081253 -3.1219441 ]
 [-1.02994699 -2.96796455]
 [-3.14002924  0.09909982]
 [-3.13944098  0.11625281]
 [ 2.37492485 -2.31010056]
 [ 0.18985749 -3.13585053]
 [ 0.26325594  3.1305432 ]
 [ 1.99838765 -2.42405676]
 [-1.72353425 -2.62660124]
 [-2.50785749  1.71543509]
 [-2.28076115 -2.16049369]
 [-0.72606345 -3.05653992]
 [-2.81317706 -1.39844172]
 [ 0.22316519  3.13365628]
 [ 2.70724173 -0.0918097 ]
 [ 1.32807457 -2.84707259]
 [-3.03309505 -0.81849789]
 [ 1.17678862  2.91286336]
 [ 2.25853354  2.18371945]
 [ 2.01695054  2.45818236]
 [ 2.58985281  1.77827637]
 

In [134]:
from sklearn.ensemble import RandomForestClassifier #using scikitlearn 0.17.1
rfc_ps = RandomForestClassifier(n_estimators = 100,random_state=42, criterion="entropy"
                                , max_features = 10
                                #, max_leaf_nodes=1000
                                #, min_samples_leaf=20
                               )

#need to handle categorical variables either through pandas.getDummies or http://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder
#train_X_num

#print(train_X_dummied.head(10))
#print(list(train_X_dummied.columns.values))



rfc_ps = rfc_ps.fit(train_X_dummied, train_y_PS_dummied)
rfc_ps_pred = rfc_ps.predict(test_X_dummied)

from sklearn.metrics import confusion_matrix

print('LifeStage Training: ', rfc_ps.score(train_X_dummied, train_y_PS_dummied))
print('LifeStage Testing: ', rfc_ps.score(test_X_dummied, test_y_PS_dummied))
confusion_matrix(test_y_PS_dummied, rfc_ps_pred, labels=["LA", "MM", "UM"])


('LifeStage Training: ', 1.0)
('LifeStage Testing: ', 0.47396504139834406)


array([[ 348, 1022,   91],
       [ 409, 1908,  250],
       [ 155,  932,  320]])

In [ ]:
#requires sklearn 0.18
#conda update sklearn

from sklearn.neural_network import MLPClassifier
ann = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
ann = ann.fit(X_train, y_train)
ann.score(X_test,y_test)

In [9]:
from sklearn.neighbors import KNeighborsClassifier

nn = KNeighborsClassifier(n_neighbors=1)
nn = nn.fit(X_train,y_train)
#nn_pred = nn.pred(X_test)
nn.score(X_test,y_test)

0.67114788004136505

In [ ]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)
dt.score(X_test,y_test)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100)
ada = ada.fit(X_train, y_train)
ada.score(X_test,y_test)

In [1]:
from sompy.sompy import SOMFactory #from https://github.com/sevamoo/SOMPY


In [ ]:
#random useful code snippets
list(data.columns.values)
data['SPEND'].dtype
data_cross.head(10)